In [10]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
# Additional libraries for data handling, preprocessing, etc.
import cv2
import os
import numpy as np

In [11]:
# Define input shape based on your image size (here, 640x360)
input_shape = (360, 640, 3)

# Load MobileNet, excluding the final classification layer
mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the pre-trained layers to avoid retraining them (optional)
for layer in mobilenet.layers:
  layer.trainable = False


C:\Users\ukrst\AppData\Local\Temp\ipykernel_6972\1475731864.py:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)


In [12]:
# GlobalAveragePooling2D reduces spatial dimensions to a single vector
x = GlobalAveragePooling2D()(mobilenet.output)

# Dense layer with 4 units (one for each variant) and softmax activation
predictions = Dense(2, activation='sigmoid')(x)

# Create the final model
model = tf.keras.Model(inputs=mobilenet.input, outputs=predictions)


In [13]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [14]:

X = []
y = []

# Folder path (replace 'path/to/images' with your actual folder path)
folder_path = 'classification_data/'

# Loop through folders
for folder_name in ['attack_img', 'explore_img']: #, 'loot_img', 'getaway_img']:
    folder_full_path = os.path.join(folder_path + folder_name)
    
    class_label = {
      'attack_img': [1,0],
      'explore_img': [0,1],
      # 'loot_img': [0,0,1,0],
      # 'getaway_img': [0,0,0,1]
    }[folder_name]
    
    for filename in os.listdir(folder_full_path):
        if filename.endswith(".jpg"):
            img = cv2.imread(os.path.join(folder_full_path, filename))

            X.append(img)
            
            y.append(class_label)
        

# X = np.asarray(X)
# y = np.asarray(y)

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
# (print(y_train[-100:-1]))

(1330, 360, 640, 3) (1330, 2)


In [18]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.7006 - loss: 0.5781 - val_accuracy: 0.6944 - val_loss: 0.6032
Epoch 2/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - accuracy: 0.7158 - loss: 0.5568 - val_accuracy: 0.7308 - val_loss: 0.5043
Epoch 3/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 129s 2s/step - accuracy: 0.7360 - loss: 0.5045 - val_accuracy: 0.7564 - val_loss: 0.4786
Epoch 4/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 129s 2s/step - accuracy: 0.7599 - loss: 0.4789 - val_accuracy: 0.7735 - val_loss: 0.4657
Epoch 5/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.7746 - loss: 0.4701 - val_accuracy: 0.7714 - val_loss: 0.4904
Epoch 6/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - accuracy: 0.7850 - loss: 0.4693 - val_accuracy: 0.7799 - val_loss: 0.4491
Epoch 7/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.7856 - loss: 0.4438 - val_accuracy: 0.7756 - val_loss: 0.4597
Epoch 8/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 0.7715 - loss: 0.4507 - val_accuracy: 0.7885 - v

In [19]:

model.save('classificator_newgen_augment.h5')

In [20]:
model_rap = tf.keras.models.load_model('classificator_newgen_augment.h5')


for n in range(20):
    y_real = y_val[n]
    tbp = np.asarray(X_val[n])
    cv2.imshow('ss', tbp)
    tbp = np.expand_dims(tbp, axis=0)
    predictions = model_rap.predict(tbp)
    print(predictions, "\n", y_real)

    cv2.waitKey(0) & 0xFF
cv2.destroyWindow('ss')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[0.6708853  0.25055686]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
[[0.3767434 0.5120115]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
[[0.5500963  0.24775137]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
[[0.2898398 0.5746978]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
[[0.2410164 0.5819752]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
[[0.5324173  0.40864065]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
[[0.49966562 0.28664666]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
[[0.38772038 0.36043194]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
[[0.73792064 0.13325748]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
[[0.8172183  0.14131215]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
[[0.19049138 0.48399773]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
[[0.3231224 0.6997154]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
[[0.708858   0.23050514]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
[[0.68254924 0.19707687]] 
 [1 0]
1/

In [21]:
model_rap = tf.keras.models.load_model('classificator_newgen_augment.h5')
model_rap.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_rap.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val))
model_rap.save('classificator_newgen_augment2.h5')

Epoch 1/40
59/59 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.7828 - loss: 0.4483 - val_accuracy: 0.7350 - val_loss: 0.4986
Epoch 2/40
59/59 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.8083 - loss: 0.4075 - val_accuracy: 0.7842 - val_loss: 0.4499
Epoch 3/40
59/59 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.8073 - loss: 0.4095 - val_accuracy: 0.7842 - val_loss: 0.4279
Epoch 4/40
59/59 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.8177 - loss: 0.3974 - val_accuracy: 0.7650 - val_loss: 0.4519
Epoch 5/40
26/59 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.8191 - loss: 0.4068 

KeyboardInterrupt: 

augmented data fragment

In [15]:
# X = []
# y = []

folder_path = 'augmented_data/'

for folder_name in ['attack_img', 'explore_img']: #, 'loot_img', 'getaway_img']:
    folder_full_path = os.path.join(folder_path + folder_name)
    
    class_label = {
      'attack_img': [1,0],
      'explore_img': [0,1],
      # 'loot_img': [0,0,1,0],
      # 'getaway_img': [0,0,0,1]
    }[folder_name]
    
    for filename in os.listdir(folder_full_path):
        if filename.endswith(".jpg") :
            img = cv2.imread(os.path.join(folder_full_path, filename))
        
            # Append image to X and label to y
            X.append(img)
            
            
            y.append(class_label)
ind = 0
# for img in X:
#     hsv_changed = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     cv2.imshow('hsv',hsv_changed); cv2.waitKey(1)
#     X[ind] = hsv_changed
#     ind += 1

# cv2.destroyAllWindows()

X = np.asarray(X)
y = np.asarray(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)

(1869, 360, 640, 3) (1869, 2)


In [5]:
model_rap = tf.keras.models.load_model('classificator_6_augmented.h5')
model_rap.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
model_rap.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))
model_rap.save('classificator_6_augmented_hsv.h5')

Epoch 1/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 134s 2s/step - accuracy: 0.6969 - loss: 0.6479 - val_accuracy: 0.7436 - val_loss: 0.5233
Epoch 2/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 116s 2s/step - accuracy: 0.7596 - loss: 0.4912 - val_accuracy: 0.7543 - val_loss: 0.4926
Epoch 3/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - accuracy: 0.7682 - loss: 0.4717 - val_accuracy: 0.7436 - val_loss: 0.4929
Epoch 4/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.7594 - loss: 0.4907 - val_accuracy: 0.7286 - val_loss: 0.4974
Epoch 5/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - accuracy: 0.7698 - loss: 0.4797 - val_accuracy: 0.7350 - val_loss: 0.4821
